In [1]:
# Import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm

# Load the data
days = pd.read_csv('Driver_Days.csv')
condition = pd.read_csv('Driver_Condition.csv')
#enrollment = pd.read_csv('Driver_Enrollment.csv')
days.Date = pd.to_datetime(days.Date)
condition.Driver_ExperStartDate = pd.to_datetime(condition.Driver_ExperStartDate)

In [2]:
condition

,Treated,Driver_City,Driver_ExperStartDate,Driver_ID
0,0,26,2016-03-30,1
1,1,8,2016-03-14,2
2,1,40,2016-04-04,3
3,0,7,2016-03-14,4
4,0,28,2016-04-03,5
...,...,...,...,...
245745,0,8,2016-04-30,245746
245746,1,27,2016-04-04,245747
245747,0,14,2016-04-18,245748
245748,0,6,2016-03-10,245749


In [15]:
days

,Date,Earnings_Dollars,DistanceDriven_Miles,TimeDriving_Minutes,Driver_ID
0,2016-03-04,86,69,257,1
1,2016-03-05,103,54,212,1
2,2016-03-06,185,124,414,1
3,2016-03-07,21,18,69,1
4,2016-03-08,59,50,257,1
...,...,...,...,...,...
5825778,2016-04-12,31,10,97,245750
5825779,2016-04-13,46,12,84,245750
5825780,2016-04-14,45,16,86,245750
5825781,2016-04-15,104,34,240,245750


In [49]:
selection = pd.merge(days, condition, on='Driver_ID', how='left')
selection['Post_Exp'] = np.where(selection.Date >= selection.Driver_ExperStartDate, 1, 0)
# selection.groupby('Driver_ID').agg({'Date': ['min', 'max']}).reset_index()
selection = selection.groupby(['Driver_ID', 'Driver_City', 'Treated', 'Post_Exp', 'Driver_ExperStartDate']).agg({'Date': 'count'}).reset_index()
selection['#days'] = np.where(selection['Post_Exp'] == 1, days.Date.max() - selection['Driver_ExperStartDate'], selection['Driver_ExperStartDate'] - days.Date.min()) 
selection['#days'] = selection['#days'].dt.days
selection['driving_rate'] = selection['Date'] / selection['#days']
selection.drop(['Date', 'Driver_ExperStartDate', '#days'], axis=1, inplace=True)
selection

,Driver_ID,Driver_City,Treated,Post_Exp,driving_rate
0,1,26,0,0,0.333333
1,1,26,0,1,0.176471
2,2,8,1,0,0.884615
3,2,8,1,1,0.939394
4,3,40,1,0,0.617021
...,...,...,...,...,...
406179,245748,14,0,0,0.344262
406180,245749,6,0,0,0.590909
406181,245749,6,0,1,1.000000
406182,245750,6,0,0,0.272727


In [56]:
selection['Treated'] = selection['Treated'].astype('category')
selection['Post_Exp'] = selection['Post_Exp'].astype('category')
selection['Driver_City'] = selection['Driver_City'].astype('category')
selection

,Driver_ID,Driver_City,Treated,Post_Exp,driving_rate
0,1,26,0,0,0.333333
1,1,26,0,1,0.176471
2,2,8,1,0,0.884615
3,2,8,1,1,0.939394
4,3,40,1,0,0.617021
...,...,...,...,...,...
406179,245748,14,0,0,0.344262
406180,245749,6,0,0,0.590909
406181,245749,6,0,1,1.000000
406182,245750,6,0,0,0.272727


In [57]:
sm.OLS.from_formula('driving_rate ~ Treated*Post_Exp + C(Driver_City)', data = selection).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           driving_rate   R-squared:                         nan
Model:                            OLS   Adj. R-squared:                    nan
Method:                 Least Squares   F-statistic:                       nan
Date:                Thu, 16 May 2024   Prob (F-statistic):                nan
Time:                        13:08:58   Log-Likelihood:                    nan
No. Observations:              406184   AIC:                               nan
Df Residuals:                  406151   BIC:                               nan
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                         nan        nan        nan        nan         nan         nan
Treated[T.1]                      nan        nan        nan        nan         nan         nan
Post_Exp[T.1]                     nan        nan        nan        nan         nan         nan
C(Driver_City)[T.4]               nan        nan        nan        nan         nan         nan
C(Driver_City)[T.5]               nan        nan        nan        nan         nan         nan
C(Driver_City)[T.6]               nan        nan        nan        nan         nan         nan
C(Driver_City)[T.7]               nan        nan        nan        nan         nan         nan
C(Driver_City)[T.8]               nan        nan        nan        nan         nan         nan
C(Driver_City)[T.10]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.12]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.14]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.15]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.20]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.21]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.22]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.23]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.24]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.25]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.26]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.27]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.28]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.29]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.30]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.35]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.40]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.41]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.45]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.48]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.49]              nan        nan        nan        nan         nan         nan
C(Driver_City)[T.50]              nan      

In [30]:
# Merge the data
selection = pd.merge(days, condition, on='Driver_ID', how='left')
# Probability processing
selection['Driving_Probability'] = 1
selection['Counts'] = selection.groupby(['Driver_ID'])['Driving_Probability'].transform('count') 
selection['Driving_Probability'] = selection['Counts']/days.Date.nunique()
# Date Processing
selection['Post_Exp'] = np.where(selection.Date >= selection.Driver_ExperStartDate, 1, 0)
# Drop the columns
selection.drop(['Date', 'Driver_ExperStartDate', 'Counts'], axis=1, inplace=True)
selection

,Earnings_Dollars,DistanceDriven_Miles,TimeDriving_Minutes,Driver_ID,Treated,Driver_City,Driving_Probability,Post_Exp
0,86,69,257,1,0,26,0.283333,0
1,103,54,212,1,0,26,0.283333,0
2,185,124,414,1,0,26,0.283333,0
3,21,18,69,1,0,26,0.283333,0
4,59,50,257,1,0,26,0.283333,0
...,...,...,...,...,...,...,...,...
5825778,31,10,97,245750,0,6,0.233333,1
5825779,46,12,84,245750,0,6,0.233333,1
5825780,45,16,86,245750,0,6,0.233333,1
5825781,104,34,240,245750,0,6,0.233333,1


In [15]:
# Running regression for each metric
for i in ['Earnings_Dollars', 'Driving_Probability', 'TimeDriving_Minutes']:
    
    effect = sm.OLS.from_formula(i + ' ~ Treated + Post_Exp + Treated:Post_Exp + C(Driver_City)', data = selection).fit()
    effect.summary()

    print(f'DID Coefficient for {i:20} is {round(effect.params["Treated:Post_Exp"], 4):7} /
          with p-value {round(effect.pvalues["Treated:Post_Exp"], 4):4} /
          and percentage increase is {round(effect.params["Treated:Post_Exp"]/effect.params["Intercept"]*100, 4):7}%')

DID Coefficient for Earnings_Dollars     is  0.9332 with p-value  0.0 and percentage increase is  0.7056%
DID Coefficient for Driving_Probability  is  0.0023 with p-value  0.0 and percentage increase is  0.4285%
DID Coefficient for TimeDriving_Minutes  is  2.0494 with p-value  0.0 and percentage increase is  0.6374%
